In [31]:
from langchain.chains import APIChain
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [32]:
import requests

openapi_spec_url = "https://alenos-tester001.azurewebsites.net/openapi?f=json"

def get_json_api_endpoint(url):
    response = requests.get(url)
    response.raise_for_status()  
    return response.json()

def remove_external_refs(spec):
    """
    Recursively search for $ref keys in the OpenAPI spec and remove those that do not start with '#'.
    """
    if isinstance(spec, dict):
        for key, value in list(spec.items()):
            if key == "$ref" and not value.startswith("#"):
                del spec[key]
            else:
                remove_external_refs(value)
    elif isinstance(spec, list):
        for item in spec:
            remove_external_refs(item)

raw_alenos_api_spec = get_json_api_endpoint(openapi_spec_url)
remove_external_refs(raw_alenos_api_spec)
raw_alenos_api_spec

{'components': {'parameters': {'bbox-crs': {'description': 'Indicates the EPSG for the given bbox coordinates.',
    'explode': False,
    'in': 'query',
    'name': 'bbox-crs',
    'required': False,
    'schema': {'default': 4326, 'type': 'integer'},
    'style': 'form'},
   'f': {'description': 'The optional f parameter indicates the output format which the server shall provide as part of the response document.  The default format is GeoJSON.',
    'explode': False,
    'in': 'query',
    'name': 'f',
    'required': False,
    'schema': {'default': 'json',
     'enum': ['json', 'html', 'jsonld'],
     'type': 'string'},
    'style': 'form'},
   'lang': {'description': 'The optional lang parameter instructs the server return a response in a certain language, if supported.  If the language is not among the available values, the Accept-Language header language will be used if it is supported. If the header is missing, the default server language is used. Note that providers may only s

In [33]:
alenos_api_spec = reduce_openapi_spec(raw_alenos_api_spec, dereference=True)
alenos_api_spec.endpoints

KeyError: 200

In [ ]:
endpoints = [
    (route, operation)
    for route, operations in raw_alenos_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

20

In [ ]:
import json
import tiktoken

enc = tiktoken.encoding_for_model('gpt-4')

def count_tokens(s):
    return len(enc.encode(s))

print(count_tokens(json.dumps(raw_alenos_api_spec)))
print(count_tokens(json.dumps(alenos_api_spec.endpoints)))

14577
1417
